In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from textwrap import dedent
from datetime import datetime

load_dotenv()

True

In [2]:
llm = AzureChatOpenAI(
    model="gpt-4o",
)

# llm = ChatOpenAI(
#     api_key=os.environ.get("OPENAI_KEY"),
#     model="gpt-4o"
# )


# Define the Agents
analista_de_edital = Agent(
    role="Analista de Edital",
    goal=(
        "Extrair e categorizar os trechos mais relevantes do edital SOMENTE para a categoria Aliança Educacional, incluindo, mas não se restringindo prazos, critérios financeiros e condições de participação." 
    ),
    backstory=(
        "Um especialista em editais brasileiros de subvenção econômica para inovação. Detalhista e metódico, extrai com precisão os trechos mais relevantes do edital para que a Cogmo, uma startup brasileira de IA, possa utilizar essas informações para submeter uma proposta."
    ),
    llm=llm,
    temperature=0.4,
    verbose=True,
    allow_delegation=False,
)

analista_de_requisitos = Agent(
    role="Levantador de Requisitos",
    goal=(
        "Identificar em cada trecho do material disponível se algum requisito de participação ou submissão de proposta foi mencionado."
        "Se houver algum requisito, este deve ser categorizado descrito como um item de uma lista de verificação."
    ),
    backstory=(
        "Experiente em processos de subvenção econômica, este agente é especializado em identificar e categorizar as ações e exigências necessárias para participação e submissão de uma proposta"
    ),
    llm=llm,
    temperature=0.2,
    verbose=True,
    allow_delegation=False,
)

supervisor = Agent(
    role="Supervisor",
    goal=(
        "O supervisor é responsável por revisar o trabalho dos agentes anteriores e garantir que todas as informações relevantes tenham sido extraídas e categorizadas corretamente."
        "Ao final, este agente deve compilar um resumo abrangente dos documentos de licitação, incluindo um resumo geral e uma lista de verificação dos requisitos para submissão de proposta pela Cogmo."
    ),
    backstory=(
        "Um líder experiente em processos de subvenção econômica, este agente é especializado em avaliar e compilar informações de editais para garantir que a Cogmo possa submeter propostas de forma eficaz."
    ),
    llm=llm,
    temperature=0.0,
    verbose=True,
    # allow_delegation=True,
)

/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [3]:
# Define the tasks
analise_de_edital = Task(
    description=(
        dedent("""
        Analisar o edital abaixo (delimitado por <<<>>>) para identificar, categorizar e extrair as sessões mais relevantes, sob o ponto de vista de uma startup de IA, que deseja submeter uma proposta.
        <<<{edital}>>>
        """)
    ),
    expected_output=(
        dedent("""
        Uma lista de dicionários, cada um contendo uma categoria e o trecho correspondente dos documentos de licitação, exatamente como consta dos originais.
        Os trechos devem ser categorizados em somente uma das categorias a seguir: prazos, financeiro, critérios_avaliacao, exigencias, outros
        Exemplos MERAMENTE ILUSTRATIVOS:
        'categoria': 'prazos', 'trecho': 'Avaliação das propostas: 30 dias após o encerramento do prazo de submissão.',
        'categoria': 'financeiro', 'trecho': 'O valor total disponível para subvenção é de R$ 1.000.000,00.',
        'categoria': 'exigencias', 'trecho': 'Os documentos necessários para participação são: ...'
        """)
    ),
    agent=analista_de_edital
)

analise_de_requisitos = Task(
    description=(
        dedent("""Analisar: 
        1. Cada trecho do edital levantado pelo agente Analista de Edital e identificar se algum requisito de participação ou submissão de proposta foi mencionado.
        2. O modelo de proposta abaixo (delimitado por <<<>>>) e identificar os requisitos relacionados à construção do documento de proposta.
        <<<# Modelo de Proposta\n\n{modelo_de_proposta}>>>\n
        Os requisitos devem ser descritos como itens acionáveis (contendo verbos no infinitivo) de uma lista de verificação.
        Cada requisito levantado deve ser auto-suficiente e não depender de informações de outros requisitos.
        """)
    ),
    expected_output=
    dedent("""
        Uma lista de dicionários, cada um contendo categoria, origem, transcrição do trecho correspondente e uma lista dos requisitos, quando houver.
        Abaixo, encontram-se exemplos MERAMENTE ILUSTRATIVOS, delimitados por <<<>>>:
        <<<'categoria': 'prazo', 'origem': 'edital', 'trecho': 'Avaliação das propostas: XX dias após ...', 'requisitos': [],
        'categoria': 'financeiro', 'origem': 'edital', 'trecho': 'O valor total disponível para subvenção é de R$ XX.000.000,00.', 'requisitos': [],
        'categoria': 'documentacao', 'origem': 'modelo_de_proposta', 'trecho': 'Os documentos necessários para participação são documento X, documento Y e documento Z', 'requisitos': ['apresentar documento X', 'apresentar documento Y', 'apresentar documento Z]'>>>
    """),
    agent=analista_de_requisitos,
    context=[analise_de_edital]
)

compilacao_de_resumo = Task(
    description=
        dedent("""
            Verificar se cada um dos requisitos levantados realmente está de acordo com os trechos extraídos dos documentos originais
            e compilar um resumo abrangente dos documentos de licitação, incluindo um resumo geral e uma lista de verificação dos requisitos para submissão de proposta pela Cogmo.
            Sempre que encontrar um requisito que não esteja de acordo com o trecho original, o agente deve solicitar a revisão daquele item ao Analista de Requisitos".
            """),
    expected_output=
        dedent("""
            Resposta final (Final Answer) em formato markdown, divido em 3 tópicos principais:
               1) GLOSSÁRIO: Contendo apenas expressões ou palavras que demandam conhecimento do contexto do edital para serrem compreendidas.
               2) DESCRITIVO: Breve resumo dos trechos mais relevantes, organizados por tópico
               3) CHECKLIST: Lista de todos os requisitos para submissão da proposta, agrupados por assunto.
            Abaixo, encontra-se um BREVE EXEMPLO (delimitado por <<<>>>) do FORMATO DESEJADO para o resultado final:
            <<<# GLOSSÁRIO
               - Macroentregas: primeiro nível da EAP do projeto, a ser entregue com a proposta
               - Contrapartida financeira: ...
               - Contrapartida econômica: ...
                        
            ## Prazos
            - Data limite para submissão das propostas: DD/MM/AAAA
            - Avaliação das propostas: XX dias após ...
            ...
                        
            ## Financeiro
            - O valor total disponível para subvenção é de R$ X.000.000,00.
            - As startups participantes devem dar uma contrapartida de 0,0Y do valor total do projeto
            ...
                        
            # Checklist
            - [ ] Documentação   
                - [ ] Apresentar documento X
                - [ ] Apresentar documento Y
                - [ ] Apresentar documento Z no modelo fornecido nos anexos do edital
            - [ ] Proposta
               - [ ] Descrever a startup proponente
                - [ ] Histórico da empresa
                - [ ] Projetos anteriores
            - [ ] Financeiro
               ...>>>
            IMPORTANTE:
            - O exemplo fornecido é meramente ilustrativo e não deve ser utilizado como parte do resultado final.
            - Espera-se que o resultado final seja mais detalhado e completo, de acordo com o edital e o modelo de proposta fornecidos.
            """),
    agent=supervisor,
    output_file = f'summary_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.md',
    # human_input=True,
    context=[analise_de_requisitos]
)

In [4]:
# Assemble the crew
crew = Crew(
    agents=[analista_de_edital, analista_de_requisitos, supervisor],
    tasks=[analise_de_edital, analise_de_requisitos, compilacao_de_resumo],
    verbose=2,
    # memory=True
)

In [5]:
# Read multiple Markdown documents
with open("/Users/morossini/Projects/Prototypes/Cogmo/LicitaBot/tender_summary_crew/docs/edital.md", "r") as file:
    edital = file.read()
with open("/Users/morossini/Projects/Prototypes/Cogmo/LicitaBot/tender_summary_crew/docs/modelo_proposta.md", "r") as file:
    modelo_proposta = file.read()
documents = {"edital": edital, "modelo_de_proposta": modelo_proposta}
print(documents)

{'edital': '# PLATAFORMA INOVAÇÃO PARA A INDÚSTRIA (Mobilização 2024)\n\n## 1. Introdução\n\nA Plataforma Inovação para a Indústria – iniciativa do Serviço Nacional de Aprendizagem Industrial (SENAI) e Serviço Social da Indústria (SESI) – tem como objetivo financiar o desenvolvimento de soluções inovadoras para a indústria brasileira, sejam elas novos produtos, processos ou serviços de caráter inovador – incremental ou radical – que promovam o aumento da produtividade e competitividade industrial brasileira, ou ainda que promovam a otimização da segurança e saúde na indústria.\n\nA Plataforma Inovação para a Indústria está correlacionada diretamente com as missões do SENAI e SESI - integrantes do Sistema Indústria -, ambos presentes nos 26 Estados e no Distrito Federal. Destaca-se que:\n\n- O SESI vem se aperfeiçoando, inovando e criando programas e projetos nas áreas de segurança e saúde no trabalho e educação. Por mais de 60 anos, tem se dedicado a atender às demandas da indústria co

In [6]:
result = crew.kickoff(inputs=documents)

 [2024-08-07 16:12:18][DEBUG]: == Working Agent: Analista de Edital
 [2024-08-07 16:12:18][INFO]: == Starting Task: 
Analisar o edital abaixo (delimitado por <<<>>>) para identificar, categorizar e extrair as sessões mais relevantes, sob o ponto de vista de uma startup de IA, que deseja submeter uma proposta.
<<<# PLATAFORMA INOVAÇÃO PARA A INDÚSTRIA (Mobilização 2024)

## 1. Introdução

A Plataforma Inovação para a Indústria – iniciativa do Serviço Nacional de Aprendizagem Industrial (SENAI) e Serviço Social da Indústria (SESI) – tem como objetivo financiar o desenvolvimento de soluções inovadoras para a indústria brasileira, sejam elas novos produtos, processos ou serviços de caráter inovador – incremental ou radical – que promovam o aumento da produtividade e competitividade industrial brasileira, ou ainda que promovam a otimização da segurança e saúde na indústria.

A Plataforma Inovação para a Indústria está correlacionada diretamente com as missões do SENAI e SESI - integrantes d

In [7]:
summary_path = "/Users/morossini/Projects/Prototypes/Cogmo/LicitaBot/tender_summary_crew/summary.md"

# Check if the file exists, if not, create it
if not os.path.exists(summary_path):
    with open(summary_path, "w") as f:
        pass

# Save the final output
with open(summary_path, "w") as f:
    f.write(str(result))

In [8]:
# Print the result
print(f"Complete results: \n{str(result)}")

Complete results: 
**

# GLOSSÁRIO
- **Macroentregas**: primeiro nível da EAP do projeto, a ser entregue com a proposta.
- **Contrapartida financeira**: contribuição financeira da empresa proponente.
- **Contrapartida econômica**: contribuição econômica da empresa proponente.
- **TRL (Technology Readiness Level)**: nível de maturidade tecnológica da solução proposta.

## DESCRITIVO

### Documentação
- **Empresa Proponente**: Informações a serem incluídas na proposta, tais como nome, CNPJ, porte, setor, site, contato (nome, função, e-mail, telefone), endereço e CNAE principal.
- **Histórico de Revisões**: Deve incluir responsável e data da revisão, e apresentar somente a última versão.
- **Apresentação da Empresa**: Descrição resumida da empresa.
- **Justificativa / Motivação**: Visão geral do contexto do projeto e problema ou oportunidade que justifica o projeto.
- **Objetivos**: Definição dos objetivos gerais e específicos, assegurando coerência com requisitos e grupos de entrega.
- *